https://huggingface.co/dragonkue/BGE-m3-ko

https://huggingface.co/kakaocorp/kanana-1.5-8b-base

# 전체 코드 작성

### 1. 환경 설정 및 라이브러리 설치

In [1]:
# ==============================================================================
# 1. 환경 설정 및 라이브러리 설치
# ==============================================================================

# 필요한 라이브러리 설치
!pip install -q datasets sentence-transformers chromadb transformers torch accelerate bitsandbytes pandas langchain_community langchain-core

import os
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import Dict, Any, List
import gc
import warnings
warnings.filterwarnings("ignore")

### 2. 데이터 로드 및 전처리

In [2]:
# ==============================================================================
# 2. 전역 설정 및 모델 정보
# ==============================================================================

# --- Global Configurations ---
LLM_MODEL_NAME = "kakaocorp/kanana-1.5-8b-instruct-2505"
EMBEDDING_MODEL_NAME = "dragonkue/bge-m3-ko"
CHROMA_DB_PATH = "./chroma_db_korquad_rag"

# --- LLM Quantization Configuration (4-bit) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- Global Variables for Loaded Resources ---
tokenizer = None
model = None
unified_db = None

# --- Few-Shot/RAG Prompt Templates ---

# Few-Shot 예시 템플릿
example_template = """
[예시 질문]: {question}
[예시 출처]: {context}
[예시 답변]: {answer}
"""
example_prompt = PromptTemplate(input_variables=["question", "answer", "context"], template=example_template)

# 최종 RAG 프롬프트 템플릿
final_rag_template_fewshot = """
## 임무:

제공된 '최종 출처' 정보만을 사용하여 '최종 질문'에 가장 정확하게 답변하십시오. 출처에 답이 없으면 '주어진 정보로는 답을 찾을 수 없습니다.'라고 답하세요. 답변은 반드시 출처에 명시된 용어로 작성하십시오.

## 학습 예시:
{few_shot_examples}

## 최종 출처:
{context}

## 최종 질문:
{question}

[최종 답변]:
"""

### 3. 임베딩 모델 및 LLM 로드

In [3]:
# ==============================================================================
# 3. 데이터 처리 및 벡터 DB 구축 함수
# ==============================================================================

def setup_database():
    """KorQuAD 데이터를 로드하고 ChromaDB를 구축 또는 로드합니다."""
    global unified_db

    if os.path.exists(CHROMA_DB_PATH):
        print(f"✅ 기존 DB 로드: {CHROMA_DB_PATH}")
        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
        unified_db = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
        return

    print("DB 구축 시작: KorQuAD 데이터 로드 및 임베딩...")

    data = load_dataset('squad_kor_v1', split='train')
    df = pd.DataFrame(data)

    full_context_data = []
    for index, row in df.iterrows():
        try:
            full_context_data.append({
                "question": row['question'],
                "answer": row['answers']['text'][0],
                "context": row['context'],
                "title": row['title']
            })
        except:
            continue

    processed_df = pd.DataFrame(full_context_data)
    rag_df = processed_df.drop_duplicates(subset=['question', 'context'])

    rag_docs = []
    for index, row in rag_df.iterrows():
        rag_docs.append(
            Document(
                page_content=row['context'],
                metadata={
                    "question": row['question'],
                    "answer": row['answer'],
                    "title": row['title']
                }
            )
        )

    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    unified_db = Chroma.from_documents(
        rag_docs,
        embedding=embeddings,
        persist_directory=CHROMA_DB_PATH
    )
    unified_db.persist()
    print("✅ 원문 Context 기반 통합 DB 구축 완료.")



### 4. 벡터 데이터베이스(ChromaDB) 구축

In [4]:
# ==============================================================================
# 4. LLM 로드 및 Tokenizer 설정
# ==============================================================================

def load_models():
    """LLM 및 Tokenizer를 로드하고 설정합니다. (중복 로드 방지)"""
    global tokenizer, model

    if model is not None:
        print("✅ 모델이 이미 로드되어 있습니다. 로딩을 건너뜜.")
        return

    print(f"LLM 로드 중: {LLM_MODEL_NAME}")

    try:
        # 1. Tokenizer 설정 (제공된 모델 설정값 반영)
        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
        tokenizer.bos_token_id = 128000
        tokenizer.eos_token_id = 128009
        tokenizer.pad_token_id = 128001

        # 2. LLM 로드 (4-bit 양자화)
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto"
        )
        model.eval()
        print("✅ LLM 로드 및 설정 완료.")
    except Exception as e:
        print(f"❌ LLM 로드 실패: {e}")
        raise RuntimeError("LLM 로드 실패. Colab 환경 (GPU) 및 라이브러리 설치를 확인하세요.")



### 5. RAG 모델 설정 및 실행

In [12]:
# ==============================================================================
# 5. RAG 추론 함수 (Few-Shot RAG)
# ==============================================================================

def few_shot_rag_invoke(query: str, k_fewshot: int = 1, k_rag: int = 4) -> Dict[str, Any]:
    """Few-Shot 예시 컨텍스트를 최종 출처에 포함하여 RAG 추론을 실행합니다."""

    if unified_db is None or model is None:
        raise RuntimeError("RAG 시스템이 초기화되지 않았습니다.")

    print(f"\nRAG 실행: 질문='{query}' (Few-Shot K={k_fewshot}, RAG K={k_rag} 문서 검색)")

    # 1. Few-Shot & RAG Context 통합 검색 및 중복 제거
    search_results = unified_db.similarity_search(query, k=k_fewshot + k_rag + 5)

    unique_results = []
    seen_content = set()
    for doc in search_results:
        if doc.page_content not in seen_content:
            unique_results.append(doc)
            seen_content.add(doc.page_content)

    # 검색 결과 분할: Few-Shot 예시와 RAG Context
    few_shot_results = unique_results[:k_fewshot]
    rag_docs = unique_results[k_fewshot:k_fewshot + k_rag]

    # 2. Few-Shot 예시 텍스트 구성 (모델 학습용)
    few_shot_examples_text = "".join([
        example_prompt.format(
            question=doc.metadata['question'],
            answer=doc.metadata['answer'],
            context=doc.page_content
        ) for doc in few_shot_results
    ])

    # 3. RAG Context 구성 (최종 출처: Few-Shot 컨텍스트 + RAG 컨텍스트)

    # Few-Shot 예시 문서의 컨텍스트를 가져옴
    few_shot_contexts = [doc.page_content for doc in few_shot_results]
    # RAG 문서의 컨텍스트를 가져옴
    rag_contexts = [doc.page_content for doc in rag_docs]

    # 두 리스트를 합쳐 최종 컨텍스트 리스트를 만들고 병합
    all_contexts = few_shot_contexts + rag_contexts
    rag_context = "\n---\n".join(all_contexts)

    # 4. 최종 프롬프트 Content 구성 (Few-Shot 템플릿 사용)
    final_prompt_content = final_rag_template_fewshot.format(
        few_shot_examples=few_shot_examples_text,
        context=rag_context,
        question=query
    )

    # 5. LLM 입력 준비 및 추론
    messages = [{"role": "user", "content": final_prompt_content}]
    inputs = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128, do_sample=False, eos_token_id=tokenizer.eos_token_id
        )

    # 6. 답변 디코딩 및 클리닝
    decoded_answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True).strip()
    clean_answer = decoded_answer
    stop_tokens = ["assistant", "[최종 답변]:", "user:"]
    for token in stop_tokens:
        if token in clean_answer:
            clean_answer = clean_answer.split(token)[0].strip()

    # 7. 최종 결과 포맷팅 및 출처 표기 추가
    source_citation_list = []

    # Few-Shot 예시 문서 포맷팅
    for i, doc in enumerate(few_shot_results):
        # 문서 제목: 유사 질문 (Few-Shot 예시) 형식
        citation_text = f"[{i+1}] {doc.metadata.get('title', '정보 없음')}: {doc.metadata.get('question', '질문 없음')}"
        source_citation_list.append(citation_text)

    # RAG Context 문서 포맷팅
    rag_doc_start_index = len(few_shot_results)
    for i, doc in enumerate(rag_docs):
        # 문서 제목: 질문 형식
        citation_text = f"[{i + rag_doc_start_index + 1}] {doc.metadata.get('title', '정보 없음')}: {doc.metadata.get('question', '질문 없음')}"
        source_citation_list.append(citation_text)

    # LLM 답변과 출처 목록을 결합
    final_answer_with_citation = clean_answer
    if source_citation_list:
        final_answer_with_citation += "\n\n**-- 참조된 위키피디아 문서 --**\n" + "\n".join(source_citation_list)

    return {
        "query": query,
        "answer": final_answer_with_citation,
        "few_shot_examples": few_shot_results,
        "retrieved_context": rag_docs
    }



### VRAM 강제 해제 함수

In [6]:
# ==============================================================================
# 6. VRAM 강제 해제 함수
# ==============================================================================

def cleanup_vram():
    """전역 변수 모델과 토크나이저를 삭제하고 GPU 캐시를 비워 VRAM을 강제로 해제합니다."""
    global model, tokenizer
    print("\n--- 🧹 VRAM 해제 시작 (공격적 해제) ---")

    if 'model' in globals() and model is not None:
        print(f"모델 객체 ({LLM_MODEL_NAME}) 삭제...")
        del model
        model = None
    if 'tokenizer' in globals() and tokenizer is not None:
        print("Tokenizer 객체 삭제...")
        del tokenizer
        tokenizer = None

    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

    print("--- ✅ VRAM 해제 완료 ---")


### 메인 실행

In [13]:
# ==============================================================================
# 7. 메인 실행 블록
# ==============================================================================

def extract_answer_sentence(context, answer):
    """정답 텍스트를 포함하는 문장을 추출합니다. (출력 가독성용)"""
    if not answer or answer not in context:
        return context[:150] + "..."

    try:
        start_index = context.find(answer)
        if start_index == -1: return context[:150] + "..."

        sentence_start = 0
        terminal_punctuations = ['.', '?', '!']

        for punc in terminal_punctuations:
            last_punc_index = context.rfind(punc, 0, start_index)
            if last_punc_index > sentence_start:
                sentence_start = last_punc_index + 1

        sentence_end = -1
        for punc in terminal_punctuations:
            punc_index = context.find(punc, start_index + len(answer))
            if punc_index != -1 and (sentence_end == -1 or punc_index < sentence_end):
                sentence_end = punc_index + 1

        if sentence_end != -1:
            snippet = context[sentence_start:sentence_end].strip()
        else:
            snippet = context[sentence_start:].strip()

        if len(snippet) > 500:
            return snippet[:500] + "..."

        return snippet

    except Exception:
        return context[:150] + f"... (추출 오류)"


if __name__ == "__main__":

    # 1. DB 및 모델 로드
    setup_database()
    load_models()

    print("\n" + "="*50)
    print("Few-Shot RAG 시스템 실행 테스트")
    print("="*50)

    # 실행 질문
    query_1 = "이순신 장군님이 사망한 전쟁의 이름은?"

    # 2. RAG 실행 (Few-Shot K=4, RAG Context K=1)
    try:
        outputs_1 = few_shot_rag_invoke(query_1, k_fewshot=4, k_rag=1)
    except RuntimeError as e:
        print(f"\n❌ 실행 오류: {e}")
        cleanup_vram()
        exit()

    # 3. 최종 결과 출력
    print("\n\n" + "="*50)
    print("Few-Shot RAG 최종 결과")
    print("="*50)

    print(f"➡️ 질문: {outputs_1['query']}")
    print(f"✅ 답변: {outputs_1['answer']}")

    print("\n--- Few-Shot 예시 (질문 기반 검색) ---")

    # Few-Shot 예시 출력
    for i, ex in enumerate(outputs_1['few_shot_examples']):
        answer = ex.metadata.get('answer', '')
        snippet = extract_answer_sentence(ex.page_content, answer)

        print(f"  {i+1}. 유사 질문: {ex.metadata['question']}")
        print(f"     문서 정답: {answer}")
        print(f"     컨텍스트 (추출): {snippet}")

    print("\n--- RAG 컨텍스트 (답변 기반 검색) ---")

    # RAG 컨텍스트 출력
    for i, doc in enumerate(outputs_1['retrieved_context']):
        answer = doc.metadata.get('answer', '')
        snippet = extract_answer_sentence(doc.page_content, answer)

        print(f"  {i+1}. 유사 질문: {doc.metadata.get('question', '정보 없음')}")
        print(f"     문서 정답 (참고): {answer}")
        print(f"     내용 (추출): {snippet}...")

    # 4. 테스트 완료 후 VRAM 정리
    cleanup_vram()


✅ 기존 DB 로드: ./chroma_db_korquad_rag
LLM 로드 중: kakaocorp/kanana-1.5-8b-instruct-2505


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ LLM 로드 및 설정 완료.

Few-Shot RAG 시스템 실행 테스트

RAG 실행: 질문='이순신 장군님이 사망한 전쟁의 이름은?' (Few-Shot K=4, RAG K=1 문서 검색)


Few-Shot RAG 최종 결과
➡️ 질문: 이순신 장군님이 사망한 전쟁의 이름은?
✅ 답변: 노량해전

**-- 참조된 위키피디아 문서 --**
[1] 이순신: 이순신이 전사한 곳은?
[2] 이순신: 이순신의 장인은 누구인가?

--- Few-Shot 예시 (질문 기반 검색) ---
  1. 유사 질문: 이순신이 전사한 곳은?
     문서 정답: 노량해협
     컨텍스트 (추출): 이순신은 명나라 부총병 진린(陳璘)과 함께 1598년 음력 11월 19일 새벽부터 노량해협에 모여 있는 일본군을 공격하였고, 일본으로 건너갈 준비를 하고 있던 왜군 선단 500여 척 가운데 200여 척을 격파, 150여 척을 파손시켰다.
  2. 유사 질문: 이순신의 장인은 누구인가?
     문서 정답: 방진
     컨텍스트 (추출): 장인은 보성군수를 역임한 방진(方震)이다.

--- RAG 컨텍스트 (답변 기반 검색) ---

--- 🧹 VRAM 해제 시작 (공격적 해제) ---
모델 객체 (kakaocorp/kanana-1.5-8b-instruct-2505) 삭제...
Tokenizer 객체 삭제...
--- ✅ VRAM 해제 완료 ---
